In [1]:
from summarytools.summarytools import dfSummary
import pandas as pd
from pathlib import Path

In [2]:
data = pd.read_csv('../data.csv')
data.head()

,中 文 名,英 文 名,曾 用 名,民 族,国家地区,出生日期,出 生 地,身 高,体 重,血 型,...,毕业院校,职 业,经纪公司,微 博,代 表 作,相关明星,weibo,avatar,bio,poster
0,黄以娜,Saem|Yina,黄世美,朝鲜族,韩国,5-4,韩国首尔,170CM,45KG,B型,...,瑞草高等学校,"演员,歌手",柠檬时代（北京）文化传播有限公司,黄以娜的新浪微博,《青年旅社》《DrFeelGood》《StartAFire》《灭罪师》,NaN,http://weibo.com/u/5623157712?refer_flag=10010...,https://image.ijq.tv/201702/14/13-33-51-66-26.jpg,黄以娜，出生于韩国。在中国发展的韩国女歌手、演员。2010年加入RaNia组合，并参演街舞电...,https://image.ijq.tv/201702/14/13-36-51-93-26.jpg
1,金泰亨,Kim Tae Hyung,V,朝鲜族,韩国,1995-12-30,韩国大邱,179CM,60KG,AB型,...,NaN,"演员,歌手",BigHit Entertainment,NaN,《花郎》,朴叔俊 朴炯植,NaN,https://image.ijq.tv/201612/15/16-18-11-62-35.jpg,金泰亨，1996年出生于韩国大邱，韩国人气男子组合防弹少年团成员。2013年通过发布单曲《2...,https://image.ijq.tv/201612/15/16-27-38-55-35.jpg
2,侯梦瑶,NaN,"梦瑶,瑶瑶,萌瑶",汉族,内地,5-8,辽宁沈阳,168CM,46KG,B型,...,北京舞蹈学院,演员,NaN,侯梦瑶的新浪微博,《多情江山》《上官婉儿》,搭档王仁君 搭档高云翔 搭档郭品超 搭档袁咏仪 搭档吕良伟 搭档邱心志,http://weibo.com/u/2285334795,https://image.ijq.tv/201704/27/14-17-25-45-19.png,侯梦瑶，出生于辽宁沈阳，中国内地女演员，毕业于北京舞蹈学院。2013年，因电视剧《花木兰传奇...,https://image.ijq.tv/201704/27/14-18-36-39-19.png
3,姚笛,NaN,"笛笛,笛妹,笛子",汉族,内地,1982-3-17,桐乡市,164CM,45KG,O型,...,北京电影学院,演员,北京鑫宝源影视投资有限公司,姚笛的新浪微博,《王小米训夫记》《爱情碟中谍》《顾盼盼的小时代》《我的宝贝》《裸婚》,文章,http://weibo.com/yaodi?refer_flag=1001030101_,https://image.ijq.tv/201912/11/14-15-47-15-46.jpg,姚笛出生于浙江桐乡市，毕业于北京电影学院。因《红楼梦》和影视结缘，在新版《红楼梦》中饰演王熙...,https://image.ijq.tv/201605/31/11-57-05-12-19.png
4,李一情,NaN,NaN,汉族,内地,2010,江苏省徐州市,150CM,26KG,B型,...,邳州机关幼儿园,演员,NaN,李一情的新浪微博,《亲爱的》《美丽的秘密》《神犬小七》,NaN,http://weibo.com/u/5312966924,https://image.ijq.tv/201512/22/10-11-46-53-13.jpg,李一情，2010年，在江苏省徐州市出生，是邳州机关幼儿园的小学生，在2014年被陈可辛导演相...,NaN


In [3]:
import numpy as np
def parse_height(x):
    x = str(x).replace('CM','')
    try:
        x = int(x)
    except:
        x = np.nan
    return x

def parse_weight(x):
    x = str(x).replace('KG','')
    try:
        x = int(x)
    except:
        x = np.nan
    return x

def parse_age(x, end_date = '2020-12-31'):
    end = pd.to_datetime(end_date)
    try:
        x = pd.to_datetime(x)
        x = int((end - x).days / 365)
    except:
        x = np.nan
    return x

In [29]:
import re

def parse_sex(x):
    x = str(x)
    if re.search('女|她',x):
        return 'Female'
    elif re.search('男|他', x):
        return 'Male'
    else:
        return 'NA'

In [43]:
data['height'] = data['身　　高'].map(parse_height) / 100
data['weight'] = data['体　　重'].map(parse_weight)
data['bmi'] = (data['weight'] / data['height'].map(lambda i: i ** 2)).round(2)
data['dob'] = data['出生日期']
data['age'] = data['出生日期'].map(parse_age)
data['sex'] = data['bio'].map(parse_sex)
data['name'] = data['中\xa0文\xa0名']
data['avatar_id'] = data['name'] + '_'+data['avatar'].map(lambda i: Path(i).name)

In [44]:
# remove duplicates
data = data.groupby(['name', 'dob'], as_index=False).apply(lambda x: x.assign(
    rank=x['avatar'].rank(method='dense', ascending=False)))
data = data.loc[data['rank'] == 1,:]

In [45]:
fields = ['name', 'dob','age', 'height','weight','bmi','avatar','bio', 'sex', 'avatar_id']
chinese_celebrity = data[fields].dropna().rename(columns=str.upper)

In [46]:
dfSummary(chinese_celebrity)

No,Variable,Stats / Values,Freqs / (% of Valid),Graph,Missing
1,NAME[object],1. 李健2. 张航3. 朱茵4. 赵亮5. 罗彬6. 马睿7. 姜彤8. 王瑛瑛9. 米娜10. 王婷11. other,"3 (0.1%)3 (0.1%)2 (0.0%)2 (0.0%)2 (0.0%)2 (0.0%)2 (0.0%)2 (0.0%)2 (0.0%)2 (0.0%)5,860 (99.6%)",,0(0.0%)
2,DOB[object],1. 19942. 1982-6-283. 1987-02-224. 1983-9-35. 1995-10-86. 1988-8-217. 1990-2-218. 1988-6-169. 1986-2-2210. 1987-3-2911. other,"9 (0.2%)8 (0.1%)7 (0.1%)7 (0.1%)5 (0.1%)5 (0.1%)5 (0.1%)5 (0.1%)5 (0.1%)5 (0.1%)5,821 (99.0%)",,0(0.0%)
3,AGE[float64],Mean (sd) : 34.6 (11.0)min < med < max:5.0 < 32.0 < 120.0IQR (CV) : 12.0 (3.1),81 distinct values,,0(0.0%)
4,HEIGHT[float64],Mean (sd) : 1.7 (0.1)min < med < max:1.5 < 1.7 < 2.0IQR (CV) : 0.1 (19.0),48 distinct values,,0(0.0%)
5,WEIGHT[float64],Mean (sd) : 59.6 (17.7)min < med < max:2.0 < 60.0 < 265.0IQR (CV) : 21.0 (3.4),132 distinct values,,0(0.0%)
6,BMI[float64],Mean (sd) : 19.4 (3.9)min < med < max:0.6 < 19.0 < 66.2IQR (CV) : 4.3 (4.9),738 distinct values,,0(0.0%)
7,AVATAR[object],1. https://image.ijq.tv/201711/022. https://image.ijq.tv/201908/143. https://image.ijq.tv/201704/294. https://image.ijq.tv/201609/105. https://image.ijq.tv/201907/236. https://image.ijq.tv/201506/117. https://image.ijq.tv/201702/118. https://www.ijq.tv/file/upload9. https://image.ijq.tv/201507/1410. https://image.ijq.tv/201508/1711. other,"2 (0.0%)2 (0.0%)2 (0.0%)2 (0.0%)1 (0.0%)1 (0.0%)1 (0.0%)1 (0.0%)1 (0.0%)1 (0.0%)5,868 (99.8%)",,0(0.0%)
8,BIO[object],1. 辛鹏，1992年11月11日出生于河南，中国内地男演员，北京2. 凯蒂·佩里（Katy Perry），原名凯瑟琳·伊丽莎白·哈3. 柳俊烈，1986年9月25日出生于首尔，韩国男演员。20144. 王瑛瑛，1996年2月29日出生于北京，中国内地女演员。就读5. 李秀美（이수미），韩国女明星、歌手，汉字本名李受美，19896. 费沁源，2001年3月20日出生于上海市，中国内地流行乐女歌7. 宋在熙，韩国著名男演员，1979年12月11日出生于韩国，毕8. 达科塔·约翰逊（Dakota Johnson），1989年19. 李佳颖（Ivleyn），1983年2月14日出生于台湾省，中10. 章子怡，1979年出生于北京，毕业于中央戏剧学院，中国电影女11. other,"2 (0.0%)2 (0.0%)2 (0.0%)2 (0.0%)1 (0.0%)1 (0.0%)1 (0.0%)1 (0.0%)1 (0.0%)1 (0.0%)5,868 (99.8%)",,0(0.0%)
9,SEX[object],1. Female2. Male3. NA,"2,684 (45.6%)1,948 (33.1%)1,250 (21.3%)",,0(0.0%)
10,AVATAR_ID[object],1. 柳俊烈_14-35-24-96-25.png2. 王瑛瑛_14-38-37-35-39.jpg3. 辛鹏_14-20-36-59-19.png4. 达科塔·约翰逊_09-55-14-13-65.png5. 安淇尔_11-10-07-66-19.png6. 史超_09-37-06-34-14.png7. 张佳泽_16-25-44-92-47.png8. 陈木易_14-33-32-89-39.jpg9. 吕星辰_15-20-38-48-26.jpg10. 朱元冰_16-38-11-40-26.jpg11. other,"2 (0.0%)2 (0.0%)2 (0.0%)1 (0.0%)1 (0.0%)1 (0.0%)1 (0.0%)1 (0.0%)1 (0.0%)1 (0.0%)5,869 (99.8%)",,0(0.0%)


In [47]:
chinese_celebrity.to_csv('../chinese-celebrity-faces.csv', index = False)